In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
from collections import Counter
from datetime import datetime
from pprint import pprint
import math
import random
import json

### Scan for pnr
#\b\d{12}\b|\d{8}-\w{4}

In [ ]:
filePath = "..\\..\\Files\\"

In [ ]:
reloadDataFiles = True

# Import files

In [ ]:
def LoadDrFile(initials):
    drDfVisists = pd.read_excel(f"{filePath}{initials}_besök.xlsx")
    drDfPhone = pd.read_excel(f"{filePath}{initials}_telefon.xlsx")
    drDfAdmin = pd.read_excel(f"{filePath}{initials}_indirekt.xlsx")
    drDf = drDfVisists.append(drDfPhone, ignore_index=True)
    drDf = drDf.append(drDfAdmin, ignore_index=True)
    limitColsDf = drDf.loc[:, ["Personnr", "Variabelnamn i M4", "Besöksdatum"] ]
    nonNaNDf = limitColsDf.loc[limitColsDf['Personnr'].notnull()]
    nonNaNDf["Initials"] = initials
    return nonNaNDf

In [ ]:
if reloadDataFiles:
    JK = LoadDrFile("JK")
    MT = LoadDrFile("MT")
    LW = LoadDrFile("LW")
    MN = LoadDrFile("MN")
    MS = LoadDrFile("MS")
    RA = LoadDrFile("RA")
    QR = LoadDrFile("QR")

In [ ]:
if reloadDataFiles:
    listingOriginalDf = pd.read_excel(f"{filePath}Listing_AFTER_COMPLETION.xlsx")
    listingDf = listingOriginalDf.loc[:, ["Pnr", "Vårdgivare"]]
    listingDf.head(2)

In [ ]:
def read_tab_separated_file(file_path):
    result_dict = {}
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Ignorera tomma rader
                if line.strip():
                    key, value = line.strip().split('\t')
                    result_dict[key] = value
    except FileNotFoundError:
        print(f"Filen {file_path} kunde inte hittas.")
    except ValueError:
        print("Fel format i filen. Kontrollera att varje rad innehåller exakt en tab.")

    return result_dict


In [ ]:
DrToInitials = read_tab_separated_file(filePath+"Permanent\\DrToInitials_ListingAnalysisAfterCompletion.txt")